In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
driver = webdriver.Chrome("../../../chromedriver")
airlines = pd.read_csv("../data/airline_links_tripadvisor.csv")

In [3]:
#Go through all the pages of reviews
for airline in airlines["airlines"]:
    print(airline)
    airline_name = []
    reviews = []
    dates = []
    scores = []
    itineraries = []
    regions = []
    classes = []
    #Get reviews from airline
    url = "https://www.tripadvisor.com" + airlines.loc[airlines['airlines'] == airline]["links"].values[0]
    driver.get(url)
    #Scroll down page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    page = BeautifulSoup(driver.page_source)
    try:
        total_pages = int(page.find("div", {"class": "pageNumbers"}).find_all("a")[-1].text)
    except:
        total_pages = 1
    for i in tqdm(range(0, total_pages, 10)):
        driver.get(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        page = BeautifulSoup(driver.page_source)

        review_blocks = page.find_all("div", {"class": "Dq9MAugU T870kzTX LnVzGwUB"})

        for review_block in review_blocks:
            #Review text
            try:
                review = review_block.find("q", {"class": "IRsGHoPm"})
                reviews.append(review.text)
            except:
                reviews.append("")
            
            #Review date
            try:
                date = review_block.find("span", {"class": "_34Xs-BQm"})
                dates.append(date.text)
            except:
                dates.append("")

            #Review scores
            try:
                score = review_block.find("div", {"class": "nf9vGX55"})
                scores.append(int(score.find("span").get("class")[1][7]))
            except:
                scores.append(-1)

            #Trip information
            try:
                tag = review_block.find("div", {"class": "hpZJCN7D"})
                tags = tag.find_all("div", {"class": "_3tp-5a1G"})
                itineraries.append(tags[0].text)
                regions.append(tags[1].text)
                classes.append(tags[2].text)
            except:
                itineraries.append("")
                regions.append("")
                classes.append("")
                
            
        if total_pages > 1:
            url = url[:60] + "or" + str((i + 1)*5) + "-" + url[60:]
        
    reviews = pd.DataFrame({"reviews": reviews, "scores": scores, "itineraries": itineraries, "regions": regions, "classes": classes})
    reviews.to_csv("../data/trip_advisor_reviews_" + airline + ".csv", sep = ",", index = False)

SyntaxError: invalid syntax (<ipython-input-3-1736c2d54fde>, line 35)